In [1]:
#importing a graph structure and other ontology URIs
from rdflib import Graph
#import other libraries
from rdflib import Graph, Literal, RDF, RDFS, URIRef, OWL, Namespace

In [2]:
#loading th KG schema
myGraph=Graph()
myGraph.parse("KG_Schema.ttl")

<Graph identifier=N85e527370c6c41c0a0f3a678e2aac0c5 (<class 'rdflib.graph.Graph'>)>

In [3]:
#number of graph elements
print(f"Number of elements:{len(myGraph)}")

Number of elements:29


In [4]:
#binding namespace prefixes
##QCL Ontology
namespace_qcl_onto = Namespace("https://github.com/DeperiasKerre/qcl_Onto/blob/main/qclontology/version-1.0/qclonto.owl#")
myGraph.bind("QpOnto", namespace_qcl_onto)
#BIBO ontology
namespace_bibo=Namespace("https://dcmi.github.io/bibo/#:")
myGraph.bind("bibo", namespace_bibo)
##MDO ontology
namespace_mdo=Namespace("https://w3id.org/mdo/core/")
myGraph.bind("mdo", namespace_mdo)
##QUDT-The QUDT ontology has several subdivisions of vocabularies
###Units
namespace_qudt_units = Namespace("https://qudt.org/vocab/unit/")
myGraph.bind("QUDT_units", namespace_qudt_units)
###Quantity Kinds
namespace_qudt_quantity_kind = Namespace("https://qudt.org/vocab/quantitykind/")
myGraph.bind("QUDT_quantity_kinds", namespace_qudt_quantity_kind)
###properties
namespace_qudt_properties = Namespace("https://qudt.org/schema/qudt/")
myGraph.bind("QUDT_properties", namespace_qudt_properties)

In [5]:
#URIS for KG Concepts

##agent
agent=URIRef("https://www.w3.org/TR/prov-o/#Agent")

##academic article
academic_article=namespace_bibo["AcademicArticle"]

##property
Property=namespace_mdo["Property"]

##laser working mode
working_mode=namespace_qcl_onto["LaserWorkingMode"]

##optoelectronic characteristics
working_temperature=namespace_qcl_onto["WorkingTemperature"]
optical_power=namespace_qcl_onto["OpticalPower"]
lasing_frequency=namespace_qcl_onto["LasingFrequency"]

##laser heterostructure
heterostructure=namespace_qcl_onto["LaserHeterostructure"]

##heterostructure materials
materials=namespace_qcl_onto["HeterostructureMaterials"]

##design type
laser_design=namespace_qcl_onto["LaserDesignType"]

##URIs for quantity Values
frequency_value=namespace_qcl_onto["FrequencyValue"]
power_value= namespace_qcl_onto["PowerValue"]
temperature_value=namespace_qcl_onto["WorkingTempValue"]

##URIs for Quantity Kinds
power=namespace_qudt_quantity_kind["Power"]
frequency=namespace_qudt_quantity_kind["Frequency"]
temperature=namespace_qudt_quantity_kind["Temperature"]

##URIS for Units
Kelvin=namespace_qudt_units["K"]
Milliwatt=namespace_qudt_units["MilliW"]
TeraHertz=namespace_qudt_units["TeraHZ"]

##Laser Working Modes
continous_wave=namespace_qcl_onto["ContinousWaveOperation"]
pulsed_mode=namespace_qcl_onto["PulsedOperation"]

##Laser Design Types
lo_phonon=namespace_qcl_onto["LOPhononDepopulation"]
bound_continum=namespace_qcl_onto["BoundToContinum"]
double_resonant=namespace_qcl_onto["DoubleResonantPhonon"]

In [6]:
#URIS for properties
#URIS for object properties
#was attributed to
attributed_to=URIRef("http://www.w3.org/ns/prov#wasAttributedTo")
#has materials
has_materials=namespace_qcl_onto["hasMaterials"]
#has quantity kind
has_quantitykind=namespace_qudt_properties["hasQuantityKind"]
#has design type
has_designtype=namespace_qcl_onto["hasDesignType"]
#relates to heterostructure
relates_to_heterostructure=namespace_qcl_onto["relatesToHeterostructure"]
#corresponds to working mode
corresponds_to_working_mode=namespace_qcl_onto["correspondsToWorkingMode"]
#has unit
has_unit=namespace_qudt_properties["hasUnit"]
#cites
cites= namespace_bibo["cites"]
#was attributed to
attributed_to=URIRef("http://www.w3.org/ns/prov#wasAttributedTo")
#has quantity value
has_quantity_value=namespace_qudt_properties["hasQuantityValue"]

#URIs for data properties
doi_prop=namespace_qcl_onto["DOI"]
materials_prop=namespace_qcl_onto["matFormula"]
numerical_value=namespace_qudt_properties["numericValue"]
publication_url=namespace_qcl_onto["URL"]

#datatypes
string=URIRef("http://www.w3.org/2001/XMLSchema#string")
number=URIRef("https://schema.org/Number")
URL=URIRef("https://schema.org/URL")

In [7]:
#Abstracts DOI and URLS
##generating paper IDS based on DOIs and linking the DOIs and the URLS for the abstracts metadata
import pandas as pd
#load abstracts
abstracts_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/abstracts/abstracts.csv")

In [8]:
#generate paper IDS based on the DOI
qcl_onto_URI="https://github.com/DeperiasKerre/qcl_Onto/blob/main/qclontology/version-1.0/qclonto.owl#"
abstracts_df['paper_id']= qcl_onto_URI +"paper_id/" + abstracts_df['DOI'].astype(str)
#verify new data frame
#abstracts_df

In [9]:
#link paper IDS, URL and DOIs
for a,b,c in zip(abstracts_df['paper_id'], abstracts_df['URL'],abstracts_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [10]:
#processing the abstract references/citations per abstract
abstracts_paper_id_list=abstracts_df['paper_id'].tolist()
#abstract 1 paper ID
abstract1_paper_id=abstracts_paper_id_list[0]
#load references
references1_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 1.csv")
#generating paper ids for references
references1_df['paper_id']= qcl_onto_URI +"paper_id/" + references1_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references1_df['paper_id'], references1_df['URL'],references1_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract1_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID 
    myGraph.add((URIRef(a), publication_url, Literal(b, datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [11]:
#abstract 2 paper ID
abstract2_paper_id=abstracts_paper_id_list[1]
#load references
references2_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 2.csv")
#generating paper ids for references
references2_df['paper_id']= qcl_onto_URI +"paper_id/" + references2_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references2_df['paper_id'], references2_df['URL'],references2_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract2_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string))) 

In [12]:
#abstract 3 paper ID
abstract3_paper_id=abstracts_paper_id_list[2]
#load references
references3_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 3.csv")
#generating paper ids for references
references3_df['paper_id']= qcl_onto_URI +"paper_id/" + references3_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references3_df['paper_id'], references3_df['URL'],references3_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract3_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [13]:
#abstract 4 paper ID
abstract4_paper_id=abstracts_paper_id_list[3]
#load references
references4_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 4.csv")
#generating paper ids for references
references4_df['paper_id']= qcl_onto_URI +"paper_id/" + references4_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references4_df['paper_id'], references4_df['URL'],references4_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract4_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [14]:
#abstract 5 paper ID
abstract5_paper_id=abstracts_paper_id_list[4]
#load references
references5_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 5.csv")
#generating paper ids for references
references5_df['paper_id']= qcl_onto_URI +"paper_id/" + references5_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references5_df['paper_id'], references5_df['URL'],references5_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract5_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [15]:
#abstract 6 paper ID
abstract6_paper_id=abstracts_paper_id_list[5]
#load references
references6_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 6.csv")
#generating paper ids for references
references6_df['paper_id']= qcl_onto_URI +"paper_id/" + references6_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references6_df['paper_id'], references6_df['URL'],references6_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract6_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string))) 

In [16]:
#abstract 7 paper ID
abstract7_paper_id=abstracts_paper_id_list[6]
#load references
references7_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 7.csv")
#generating paper ids for references
references7_df['paper_id']= qcl_onto_URI +"paper_id/" + references7_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references7_df['paper_id'], references7_df['URL'],references7_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract7_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [17]:
#abstract 8 paper ID
abstract8_paper_id=abstracts_paper_id_list[7]
#load references
references8_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 8.csv")
#generating paper ids for references
references8_df['paper_id']= qcl_onto_URI +"paper_id/" + references8_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references8_df['paper_id'], references8_df['URL'],references8_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract8_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [18]:
#abstract 9 paper ID
abstract9_paper_id=abstracts_paper_id_list[8]
#load references
references9_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 9.csv")
#generating paper ids for references
references9_df['paper_id']= qcl_onto_URI +"paper_id/" + references9_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references9_df['paper_id'], references9_df['URL'],references9_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract9_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [19]:
#abstract 10 paper ID
abstract10_paper_id=abstracts_paper_id_list[9]
#load references
references10_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 10.csv")
#generating paper ids for references
references10_df['paper_id']= qcl_onto_URI +"paper_id/" + references10_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references10_df['paper_id'], references10_df['URL'],references10_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract10_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [20]:
#abstract 11 paper ID
abstract11_paper_id=abstracts_paper_id_list[10]
#load references
references11_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 11.csv")
#generating paper ids for references
references11_df['paper_id']= qcl_onto_URI +"paper_id/" + references11_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references11_df['paper_id'], references11_df['URL'],references11_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract11_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [21]:
#abstract 12 paper ID
abstract12_paper_id=abstracts_paper_id_list[11]
#load references
references12_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 12.csv")
#generating paper ids for references
references12_df['paper_id']= qcl_onto_URI +"paper_id/" + references12_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references12_df['paper_id'], references12_df['URL'],references12_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract12_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [22]:
#abstract 13 paper ID
abstract13_paper_id=abstracts_paper_id_list[12]
#load references
references13_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 13.csv")
#generating paper ids for references
references13_df['paper_id']= qcl_onto_URI +"paper_id/" + references13_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references13_df['paper_id'], references13_df['URL'],references13_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract13_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [23]:
#abstract 14 paper ID
abstract14_paper_id=abstracts_paper_id_list[13]
#load references
references14_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 14.csv")
#generating paper ids for references
references14_df['paper_id']= qcl_onto_URI +"paper_id/" + references14_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references14_df['paper_id'], references14_df['URL'],references14_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract14_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [24]:
#abstract 15 paper ID
abstract15_paper_id=abstracts_paper_id_list[14]
#load references
references15_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 15.csv")
#generating paper ids for references
references15_df['paper_id']= qcl_onto_URI +"paper_id/" + references15_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references15_df['paper_id'], references15_df['URL'],references15_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract15_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [25]:
#abstract 16 paper ID
abstract16_paper_id=abstracts_paper_id_list[15]
#load references
references16_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 16.csv")
#generating paper ids for references
references16_df['paper_id']= qcl_onto_URI +"paper_id/" + references16_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references16_df['paper_id'], references16_df['URL'],references16_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract16_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [26]:
#abstract 17 paper ID
abstract17_paper_id=abstracts_paper_id_list[16]
#load references
references17_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 17.csv")
#generating paper ids for references
references17_df['paper_id']= qcl_onto_URI +"paper_id/" + references17_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references17_df['paper_id'], references17_df['URL'],references17_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract17_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [27]:
#abstract 18 paper ID
abstract18_paper_id=abstracts_paper_id_list[17]
#load references
references18_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 18.csv")
#generating paper ids for references
references18_df['paper_id']= qcl_onto_URI +"paper_id/" + references18_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references18_df['paper_id'], references18_df['URL'],references18_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract18_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [28]:
#abstract 19 paper ID
abstract19_paper_id=abstracts_paper_id_list[18]
#load references
references19_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 19.csv")
#generating paper ids for references
references19_df['paper_id']= qcl_onto_URI +"paper_id/" + references19_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references19_df['paper_id'], references19_df['URL'],references19_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract19_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [29]:
#abstract 20 paper ID
abstract20_paper_id=abstracts_paper_id_list[19]
#load references
references20_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 20.csv")
#generating paper ids for references
references20_df['paper_id']= qcl_onto_URI +"paper_id/" + references20_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references20_df['paper_id'], references20_df['URL'],references20_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract20_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [30]:
#abstract 21 paper ID
abstract21_paper_id=abstracts_paper_id_list[20]
#load references
references21_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 21.csv")
#generating paper ids for references
references21_df['paper_id']= qcl_onto_URI +"paper_id/" + references21_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references21_df['paper_id'], references21_df['URL'],references21_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract21_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [31]:
#abstract 22 paper ID
abstract22_paper_id=abstracts_paper_id_list[21]
#load references
references22_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 22.csv")
#generating paper ids for references
references22_df['paper_id']= qcl_onto_URI +"paper_id/" + references22_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references22_df['paper_id'], references22_df['URL'],references22_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract22_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [32]:
#abstract 23 paper ID
abstract23_paper_id=abstracts_paper_id_list[22]
#load references
references23_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 23.csv")
#generating paper ids for references
references23_df['paper_id']= qcl_onto_URI +"paper_id/" + references23_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references23_df['paper_id'], references23_df['URL'],references23_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract23_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [33]:
#abstract 24 paper ID
abstract24_paper_id=abstracts_paper_id_list[23]
#load references
references24_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 24.csv")
#generating paper ids for references
references24_df['paper_id']= qcl_onto_URI +"paper_id/" + references24_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references24_df['paper_id'], references24_df['URL'],references24_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract24_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [34]:
#abstract 25 paper ID
abstract25_paper_id=abstracts_paper_id_list[24]
#load references
references25_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 25.csv")
#generating paper ids for references
references25_df['paper_id']= qcl_onto_URI +"paper_id/" + references25_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references25_df['paper_id'], references25_df['URL'],references25_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract25_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [35]:
#abstract 26 paper ID
abstract26_paper_id=abstracts_paper_id_list[25]
#load references
references26_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 26.csv")
#generating paper ids for references
references26_df['paper_id']= qcl_onto_URI +"paper_id/" + references26_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references26_df['paper_id'], references26_df['URL'],references26_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract26_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [36]:
#abstract 27 paper ID
abstract27_paper_id=abstracts_paper_id_list[26]
#load references
references27_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 27.csv")
#generating paper ids for references
references27_df['paper_id']= qcl_onto_URI +"paper_id/" + references27_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references27_df['paper_id'], references27_df['URL'],references27_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract27_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [37]:
#abstract 28 paper ID
abstract28_paper_id=abstracts_paper_id_list[27]
#load references
references28_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 28.csv")
#generating paper ids for references
references28_df['paper_id']= qcl_onto_URI +"paper_id/" + references28_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references28_df['paper_id'], references28_df['URL'],references28_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract28_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [38]:
#abstract 29 paper ID
abstract29_paper_id=abstracts_paper_id_list[28]
#load references
references29_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 29.csv")
#generating paper ids for references
references29_df['paper_id']= qcl_onto_URI +"paper_id/" + references29_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references29_df['paper_id'], references29_df['URL'],references29_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract29_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [39]:
#abstract 30 paper ID
abstract30_paper_id=abstracts_paper_id_list[29]
#load references
references30_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 30.csv")
#generating paper ids for references
references30_df['paper_id']= qcl_onto_URI +"paper_id/" + references30_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references30_df['paper_id'], references30_df['URL'],references30_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract30_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [40]:
#abstract 31 paper ID
abstract31_paper_id=abstracts_paper_id_list[30]
#load references
references31_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 31.csv")
#generating paper ids for references
references31_df['paper_id']= qcl_onto_URI +"paper_id/" + references31_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references31_df['paper_id'], references31_df['URL'],references31_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract31_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [41]:
#abstract 32 paper ID
abstract32_paper_id=abstracts_paper_id_list[31]
#load references
references32_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 32.csv")
#generating paper ids for references
references32_df['paper_id']= qcl_onto_URI +"paper_id/" + references32_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references32_df['paper_id'], references32_df['URL'],references32_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract32_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [42]:
#abstract 33 paper ID
abstract33_paper_id=abstracts_paper_id_list[32]
#load references
references33_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 33.csv")
#generating paper ids for references
references33_df['paper_id']= qcl_onto_URI +"paper_id/" + references33_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references33_df['paper_id'], references33_df['URL'],references33_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract33_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [43]:
#abstract 34 paper ID
abstract34_paper_id=abstracts_paper_id_list[33]
#load references
references34_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 34.csv")
#generating paper ids for references
references34_df['paper_id']= qcl_onto_URI +"paper_id/" + references34_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references34_df['paper_id'], references34_df['URL'],references34_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract34_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [44]:
#abstract 35 paper ID
abstract35_paper_id=abstracts_paper_id_list[34]
#load references
references35_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 35.csv")
#generating paper ids for references
references35_df['paper_id']= qcl_onto_URI +"paper_id/" + references35_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references35_df['paper_id'], references35_df['URL'],references35_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract35_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [45]:
#abstract 36 paper ID
abstract36_paper_id=abstracts_paper_id_list[35]
#load references
references36_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 36.csv")
#generating paper ids for references
references36_df['paper_id']= qcl_onto_URI +"paper_id/" + references36_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references36_df['paper_id'], references36_df['URL'],references36_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract36_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [46]:
#abstract 37 paper ID
abstract37_paper_id=abstracts_paper_id_list[36]
#load references
references37_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 37.csv")
#generating paper ids for references
references37_df['paper_id']= qcl_onto_URI +"paper_id/" + references37_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references37_df['paper_id'], references37_df['URL'],references37_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract37_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [47]:
#abstract 38 paper ID
abstract38_paper_id=abstracts_paper_id_list[37]
#load references
references38_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 38.csv")
#generating paper ids for references
references38_df['paper_id']= qcl_onto_URI +"paper_id/" + references38_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references38_df['paper_id'], references38_df['URL'],references38_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract38_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [48]:
#abstract 39 paper ID
abstract39_paper_id=abstracts_paper_id_list[38]
#load references
references39_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 39.csv")
#generating paper ids for references
references39_df['paper_id']= qcl_onto_URI +"paper_id/" + references39_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references39_df['paper_id'], references39_df['URL'],references39_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract39_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [49]:
#abstract 40 paper ID
abstract40_paper_id=abstracts_paper_id_list[39]
#load references
references40_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 40.csv")
#generating paper ids for references
references40_df['paper_id']= qcl_onto_URI +"paper_id/" + references40_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references40_df['paper_id'], references40_df['URL'],references40_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract40_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [50]:
#abstract 41 paper ID
abstract41_paper_id=abstracts_paper_id_list[40]
#load references
references41_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 41.csv")
#generating paper ids for references
references41_df['paper_id']= qcl_onto_URI +"paper_id/" + references41_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references41_df['paper_id'], references41_df['URL'], references41_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract41_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [51]:
#abstract 42 paper ID
abstract42_paper_id=abstracts_paper_id_list[41]
#load references
references42_df=pd.read_csv("C:/Users/USER/Documents/Projects/KG/Data/references/abstract 42.csv")
#generating paper ids for references
references42_df['paper_id']= qcl_onto_URI +"paper_id/" + references42_df['DOI'].astype(str)
#link each reference paper ID to its DOI and URL
for a,b,c in zip(references42_df['paper_id'], references42_df['URL'], references42_df['DOI']):
    myGraph.add((URIRef(a), RDF.type, (academic_article)))
    myGraph.add((URIRef(abstract42_paper_id), cites, URIRef(a)))#map the citations with the abstract paper ID
    myGraph.add((URIRef(a), publication_url, Literal(b,datatype=URL)))
    myGraph.add((URIRef(a), doi_prop, Literal(c,datatype=string)))

In [52]:
print(f"Number of elements:{len(myGraph)}")

Number of elements:2514


In [53]:
#Serializing the generated RDF data in turtle format
kg_data_ttl = myGraph.serialize(format='turtle')
#print(xml_rdf)
# Save the serialized data to a file
with open('qcl_KG_references.ttl', 'wb') as f:
    f.write(kg_data_ttl.encode('utf-8'))

In [54]:
#serialize to rdf/xml format
kg_data_rdf = myGraph.serialize(format='xml')
#print(xml_rdf)
# Save the serialized data to a file
with open('qcl_KG_references.rdf', 'wb') as f:
    f.write(kg_data_rdf.encode('utf-8'))